In [27]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [28]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='llama-3.3-70b-versatile')

In [29]:
ai_msg = llm.invoke([
    ("system","You are a chatbot"),
    ("user", "i want help")
])
ai_msg.content

"I'm here to help. Could you please provide more details about what you need help with? Is it a specific problem, a question, or something else? I'll do my best to assist you."

In [30]:
system_message = """
You are an advanced language model designed to preprocess user inputs for an expense tracking system. Your role is to extract structured information from the input and generate a concise summary of the user's request. This summary will be used later for semantic search and database operations.

### Instructions
1. Extract the following fields based on the user's input:
   - **UserId**: {user_id}
   - **Action**: One of "CREATE", "READ", "UPDATE", or "DELETE".  
   - **Amount**: The expense amount (e.g., 50.00).  
   - **Category**: The category associated with the expense {categories}. If unspecified, use "Miscellaneous".  
   - **Date**: The date of the expense in a dd-mm-yyyy format. Calculate the date based on provided current date {current_date}. If no date is mentioned, consider a today.  
2. Create a **Notes** field that is a concise description summarizing the user's request or intent. This will help retrieve similar records using semantic search. Avoid copying raw user text verbatim. Also add the date if it is relevant to the request.

### Rules
- Only summarize the user request for **Notes** (e.g., "Add a groceries expense for 200 on January 10, 2025").
- Handle missing or incomplete information explicitly (e.g., "Amount: Not specified").
- Use predefined categories where possible; otherwise, classify as "Miscellaneous".

### User Input
User Input: "{input}"

### Output Format
Respond in plain text fields like below:

UserId: <user_id>
Action: <CRUD Action>  
Amount: <number or "Not specified">  
Category: <string>  
Date: <human-readable date or "today">  
Notes: <summary of the user's request>

### Examples

**Example 1**  
Input: "I spent 200 on groceries yesterday."  
Response:
UserId: <user_id>
Action: CREATE  
Amount: 200  
Category: Groceries  
Date: 10-01-2025
Notes: Add a groceries expense of 200 from yesterday (10-01-2025).

**Example 2**  
Input: "Find expenses for last month's travel."  
Response:  
UserId: <user_id>
Action: READ  
Amount: Not specified  
Category: Travel  
Date: 10-12-2025 
Notes: Retrieve travel expenses from last month (10-12-2025).

**Example 3**
Input: "Remove 9/1/2025 expense of 500 in entertainment."  
Response:  
UserId: <user_id>
Action: DELETE  
Amount: 500  
Category: Entertainment  
Date: 09-01-2025
Notes: Delete an entertainment expense of 500 from 09-01-2025.

**Example 4**  
Input: "How much did I spend on food last year?"  
Response:  
UserId: <user_id>
Action: READ  
Amount: Not specified  
Category: Food  
Date: 01-01-2024  
Notes: Retrieve total food expenses from last year 2024.

**Example 5**  
Input: "Add 1000 for car maintenance tomorrow."  
Response:  
UserId: <user_id>
Action: CREATE  
Amount: 1000  
Category: Car Maintenance  
Date: 12-01-2025  
Notes: Add a car maintenance expense of 1000 for tomorrow (12-01-2025).

---

Follow these instructions exactly to generate structured data and a meaningful summary for each request.
"""

In [31]:
chat = llm.invoke([
    ("system",system_message),
    ("user", "ate 20$ pizza")
])
chat.content

'UserId: {user_id}\nAction: CREATE\nAmount: 20.00\nCategory: Food\nDate: today\nNotes: Add a food expense of 20 for pizza today.'

In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_message,
        ),
        ("human", "{input}"),
    ]
)

In [33]:
chain = prompt | llm
chain.invoke(
    {
        "categories": "Groceries, Travel, Entertainment, Food, Shopping",
        "current_date": "11-01-2025",
        "input": "bought shoes for 20$ yesterday",
        "user_id": "8"
    }
).content

'UserId: 8\nAction: CREATE\nAmount: 20.00\nCategory: Shopping\nDate: 10-01-2025\nNotes: Add a shopping expense of 20 for shoes from yesterday (10-01-2025).'